In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [2]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [6]:
model.invoke("which country won the 2022 world cup?")

AIMessage(content='As an AI, I am unable to predict future events. The 2022 FIFA World Cup is scheduled to take place in Qatar from November 21 to December 18, 2022. The winner of the tournament has not been determined yet.', response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 17, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f9f71655-c875-444d-b830-094df40333b3-0', usage_metadata={'input_tokens': 17, 'output_tokens': 50, 'total_tokens': 67})

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("what is the day today?")

'Today is Wednesday.'

## RAGAS 

In [4]:
import requests
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the data
loader = PyPDFLoader('../data/Final Askuala1$2.pdf')

documents = loader.load()

# Chunk the data
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv,find_dotenv

# Load OpenAI API key from .env file
load_dotenv(find_dotenv())

# Setup vector database
client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

# Populate vector database
vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

# Define vectorstore as retriever to enable semantic search
retriever = vectorstore.as_retriever()